# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [2]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

The Heckscher-Ohlin theorem reveals the reasons why each country has a comparative advantage. They observe that: there are differences in the resources that each country possesses. For example, Argentina has a lot of fertile land, Saudi Arabia has a lot of oil.  And products can be classified according to the different factors of production required.

Based on these, they propose the H-O theorem, which states that a country will have a relative advantage in producing products that require a large use of the resources it has in abundance.

The model follows these assumptions:
- there are 2 countries to keep the model simpel
- there are 2 factors: capital and labor 
- Prices are the same everywhere
- Tastes in the 2 countries are the same, that means there are no differeneces in preferences
- There is perfect competition: firms choose outputlevel at which price = marginal costs, there is free entry and exit of firms in the market and the necessary information is available and perfect
- There are no transport costs
- There are no trade restrictions

Two-country model:

In [50]:
import numpy as np

# Define parameters:

A1 = 1    # productivity of industry 1
A2 = 1    # productivity of industry 2

L1 = 500   # total available labor of country 1
L2 = 100  # total available labour of country 2
K1 = 350  # total available capital of country 1
K2 = 200  # total available capital of country 2

r = 0.03  # interest rate
w = 1     # wage rate


alpha1 = 1/3 # capital vs labor weight ratio in production of good 1
alpha2 = 2/3 # capital vs labor weight ratio in production of good 2

beta1 = 0.5 # product 1 vs 2 weight ratio in utility for country 1
beta2 = 0.5 # product 1 vs 2 weight ratio in utility for country 2

# Define the production functions
Q1 = lambda L, K: A1 * (L ** (1 - alpha1)) * (K ** alpha1)
Q2 = lambda L_tot, L, K_tot, K: A2 * ((L_tot - L) ** (1 - alpha2)) * ((K_tot - K) ** alpha2)

# Define the utility functions 
U1 = lambda u, L, K:(Q1(L, K) ** u) * (Q2(L1, L, K1, K) ** (1 - u))
U2 = lambda u, L, K:(Q1(L, K) ** u) * (Q2(L2, L, K2, K) ** (1 - u))

"""For closed economies"""
print("Without trade:")


# Calculate best utility for country 1:

U1_max = 0

for l in range(L1):
    for k in range(K1):

        utility1 = U1(beta1  , l , k)
        if utility1 > U1_max:
            U1_max = utility1
            L_opt1 = l
            K_opt1 = k
print("Country 1 maximizes their utility at %.1f by setting labor and capital for industry 1 at %.0f and %.0f" % (U1_max , L_opt1 , K_opt1))

# Calculate best utility for country 2:

U2_max = 0

for l in range(L2):
    for k in range(K2):

        utility2 = U2(beta2 , l , k)
        if utility2 > U2_max:
            U2_max = utility2
            L_opt2 = l
            K_opt2 = k

print("Country 2 maximizes their utility at %.1f by setting labor and capital for industry 1 at %.0f and %.0f\n" % (U2_max , L_opt2 , K_opt2))

# Goods produced at given level:

Q11 = Q1(L_opt1 , K_opt1)
Q21 = Q2(L1 , L_opt1 , K1 , K_opt1)
Q12 = Q1(L_opt2 , K_opt2)
Q22 = Q2(L2 , L_opt2 , K2 , K_opt2)

print("Total production of country 1:\n Industry 1: %.1f \n Industry 2: %.1f \n\nTotal production of country 2:\n Industry 1: %.1f\n Industry 2: %.1f" % (Q11 , Q21 , Q12 , Q22))


# Marginal Utility 

#(Change in utility with one unit of labor added)
U1_diff_L = U1(beta1 , (L_opt1 + 1) , K_opt1)
U2_diff_L = U2(beta2 , (L_opt2 + 1) , K_opt2)

MRU1L = U1_diff_L - U1_max
MRU2L = U2_diff_L - U2_max

#(Change in utility with one unit of labor added)
U1_diff_K = U1(beta1 , L_opt1 , (K_opt1 + 1))
U2_diff_K = U2(beta2 , L_opt2 , (K_opt2 + 1))

MRU1K = U1_diff_K - U1_max
MRU2K = U2_diff_K - U2_max

print("\nMarginal utility:\n\n Country 1:\n  Labor: %.4f\n  Capital: %.4f \n\n Country 2:\n  Labor: %.4f\n  Capital: %.4f" % (MRU1L , MRU1K , MRU2L , MRU2K))




Without trade:
Country 1 maximizes their utility at 221.4 by setting labor and capital for industry 1 at 333 and 117
Country 2 maximizes their utility at 74.8 by setting labor and capital for industry 1 at 67 and 67

Total production of country 1:
 Industry 1: 235.0 
 Industry 2: 208.5 

Total production of country 2:
 Industry 1: 67.0
 Industry 2: 83.6

Marginal utility:

 Country 1:
  Labor: -0.0003
  Capital: -0.0034 

 Country 2:
  Labor: -0.0142
  Capital: -0.0035


In [ ]:
# With trade


# Find marginal rate of substitution:



## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

In [3]:
# Step 1: Define the production function
K, L, a, b = sm.symbols('K L a b')
Q = a * K**b * L**(1-b)

# Step 2: Calculate the factor intensities
labor_intensity = Q.diff(L) / Q
capital_intensity = Q.diff(K) / Q

# Step 3: Determine the factor endowments
K1 = 500
L1 = 1000
K2 = 1000
L2 = 500

# Step 4: Calculate the autarky prices
Q1 = Q.subs([(K, K1), (L, L1)])
Q2 = Q.subs([(K, K2), (L, L2)])

P1 = Q1 / (a * L1**b * K1**(1-b))
P2 = Q2 / (a * L2**b * K2**(1-b))

print("Autarky prices:")
print("P1:", P1)
print("P2:", P2)

# Step 5: Calculate the relative factor prices
w, r = sm.symbols('w r')
eq1 = sm.Eq(w * labor_intensity.subs([(K, K1), (L, L1), (a, a), (b, b)]),
           r * capital_intensity.subs([(K, K1), (L, L1), (a, a), (b, b)]))
eq2 = sm.Eq(w * labor_intensity.subs([(K, K2), (L, L2), (a, a), (b, b)]),
           r * capital_intensity.subs([(K, K2), (L, L2), (a, a), (b, b)]))
sol = sm.solve((eq1, eq2), (w, r))
w_star, r_star = sol[w], sol[r]
print("Relative factor prices:")
print("w/r:", w_star/r_star)

# Step 6: Determine the pattern of trade
if w_star/r_star > P1/P2:
    print("Country 1 exports the labor-intensive good")
else:
    print("Country 1 exports the capital-intensive good")

# Step 7: Calculate the equilibrium prices and quantities
Q1 = r_star * capital_intensity.subs([(K, K1), (L, L1), (a, a), (b, b)])
Q2 = w_star * labor_intensity.subs([(K, K2), (L, L2), (a, a), (b, b)])

print("Equilibrium prices:")
print("P1:", P1 + Q1)
print("P2:", P2 + Q2)
print("Equilibrium quantities:")
print("Q1:", Q1)
print("Q2:", Q2)

Autarky prices:
P1: 1000**(1 - b)*500**b*500**(b - 1)/1000**b
P2: 1000**b*1000**(b - 1)*500**(1 - b)/500**b
Relative factor prices:
w/r: nan


TypeError: Invalid NaN comparison

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 